In [21]:
# TEST init_word_representations.py MODULE
import init_word_representations as iwr

## TEST get_training_examples FUCNTION
(x_u, x_r, y, max_len) = iwr.get_training_examples("./data/webquestions.examples.train.e2e.top10.filter.patrel.sid.tsv")


ValueError: need more than 2 values to unpack

In [39]:
import numpy as np
from init_word_representations import clean_str
def get_training_examples_raw(training_file_name, n_neg=5000):
    """
    Load training data file, and split the data into words and labels.
    Return utterances, relation words, 
    labels and the largest length of training sentences.
    """
    positive_utterances = []
    positive_relations = []
    negative_utterances = []
    negative_relations = []
    max_len = 0
    with open(training_file_name) as tf:
        for lines in tf:
            # data -> [f1, u_str, r_str, WQ_NUM]
            data = lines.split('\t')
            f1 = float(data[0])
            u_str = clean_str(data[1].strip()).split(" ")
            r_str = clean_str(data[2].strip()).split(" ")
            max_len = max(len(u_str), len(r_str), max_len)
            if f1 >= 0.5:
                positive_utterances.append(u_str)
                positive_relations.append(r_str)
            elif f1 == 0:
                negative_utterances.append(u_str)
                negative_relations.append(r_str)
        samples = np.random.choice(len(negative_utterances), n_neg, replace=False)
        negative_utterances = [negative_utterances[i] for i in samples]
        negative_relations = [negative_relations[i] for i in samples]
        x_u = positive_utterances + negative_utterances
        x_r = positive_relations + negative_relations
        positive_labels = np.ones(len(positive_utterances))
        negative_labels = np.zeros(len(negative_utterances))
        y = np.concatenate((positive_labels, negative_labels))
    
    return (x_u, x_r, y, max_len)

## TEST get_training_examples FUCNTION
(x_u, x_r, y, max_len) = get_training_examples("./data/webquestions.examples.train.e2e.top10.filter.patrel.sid.tsv")


In [40]:
x_u[7100], x_r[7100], y[7100]

(['where', 'was', 'e1', 'when', 'he', 'died'],
 ['base', 'kwebbase', 'kwtopic', 'disciplines'],
 0.0)

In [48]:
import numpy as np
from init_word_representations import clean_str
def get_training_examples_for_softmax(training_file_name, n_neg_sample=5):
    """
    Load training data file, and split the data into words and labels.
    Return utterances, relation words, 
    labels and the largest length of training sentences.
    The output of this funtion is formatted for softmax regression.
    """
    neg_dict = {}
    max_len = 0
    wqn_lst = []
    x_u = []
    x_r = []
    y = []
    with open(training_file_name) as tf:
        for lines in tf:
            # data -> [f1, u_str, r_str, WQ_NUM]
            data = lines.split('\t')
            f1 = float(data[0])
            u_str = clean_str(data[1].strip()).split(" ")
            r_str = clean_str(data[2].strip()).split(" ")
            wqn = data[3].strip()
            max_len = max(len(u_str), len(r_str), max_len)
            
            if f1 >= 0.5:
                wqn_lst.append(wqn)
                x_u.append(u_str)
                x_r.append([r_str])
            else:
                if neg_dict.has_key(wqn):
                    if len(neg_dict[wqn]) < n_neg_sample:
                        neg_dict[wqn].append(r_str)
                else:
                    neg_dict[wqn] = [r_str]
        for i in range(len(x_u)):
            if not neg_dict.has_key(wqn_lst[i]):
                neg_dict[wqn_lst[i]] = neg_dict[wqn_lst[0]]
            if len(neg_dict[wqn_lst[i]]) < n_neg_sample:
                neg_dict[wqn_lst[i]] += neg_dict[wqn_lst[0]][:n_neg_sample - len(neg_dict[wqn_lst[i]])]
            if len(neg_dict[wqn_lst[i]]) != n_neg_sample:
                print neg_dict[wqn_lst[i]]
            x_r[i] = x_r[i] + neg_dict[wqn_lst[i]]
            # add a little randomness 
            y.append(np.random.randint(len(x_r[i])))
            tmp = x_r[i][0]
            x_r[i][0] = x_r[i][y[i]]
            x_r[i][y[i]] = tmp
    
    return (x_u, x_r, y, max_len)

In [49]:
(x_u, x_r, y, max_len) = get_training_examples_for_softmax("./data/webquestions.examples.train.e2e.top10.filter.patrel.sid.tsv")

In [1]:
import tensorflow as tf
import numpy as np
sess = tf.InteractiveSession()

In [20]:
a = tf.Variable([[[1.0,2.0,3.0],[4.0,5.0,6.0],[7.0,8.0,9.0]],[[1.0,2.0,3.0],[4.0,5.0,6.0],[11.0,11.0,11.0]]])
b = tf.constant([[1.0,2.0,3.0],[-1.0,2.0,-3.0]])
c = tf.reshape(b, [2, 1, 3])
a.initializer.run()

In [21]:
print tf.sqrt(tf.reduce_sum(a**2, 2)).eval()
print tf.sqrt(tf.reduce_sum(c**2, 2)).eval()
d = tf.sqrt(tf.reduce_sum(a**2, 2)) * tf.sqrt(tf.reduce_sum(c**2, 2))

print (tf.reduce_sum(tf.mul(a,c), 2) / d).eval()

[[  3.74165726   8.77496433  13.92838669]
 [  3.74165726   8.77496433  19.0525589 ]]
[[ 3.7416575]
 [ 3.7416575]]
[[ 1.          0.97463179  0.95941204]
 [-0.42857143 -0.36548692 -0.30860671]]
